<a href="https://colab.research.google.com/github/panghanwu/tibame_project/blob/main/recommendation_oop_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模糊搜尋功能 v2
---

### 會使用到的套件
- py2neo
- pandas
- numpy

In [40]:
# py2neo是python控制neo4j的套件
# Colab並未內建需要另外安裝
!pip install py2neo

In [63]:
import numpy as np
import pandas as pd
import py2neo as neo


class Neo4jRecomBot():
    # 1. __init__: 連上Neo4j伺服器
    # 2. fussy_search: 模糊搜尋
    # 3. same_search: 同款搜尋
    # 4. fit_search: 穿搭搜尋
    def __init__(self, sever_link, password, word2vec_path):
        # 載入圖資料庫
        self.graph = neo.Graph(sever_link, password=password)
        # 提取出產品清單
        self.product_list = list(neo.NodeMatch(self.graph, labels=frozenset(['Product'])))
        
        # 載入語料詞向量檔
        cloth_model_df = pd.read_csv(word2vec_path)
        cloth_vec = {}
        for i in range(len(cloth_model_df)):
            vector = cloth_model_df['vec'][i]
            vector = vector.replace('\n','')
            vector = vector.replace('[','')
            vector = vector.replace(']','')
            vector = np.fromstring(vector, sep=' ')
            cloth_vec[cloth_model_df['cht'][i]] = vector

        self.cloth_vec = cloth_vec
    

    def fussy_search(self, keyword, gender=None):
        assert gender in ['man', 'woman', None]
        
        # 依據性別更改商品節點清單
        if gender == 'man':
            search_list = [x for x in self.product_list if x['sn'][0]=='M']
        elif gender == 'woman':
            search_list = [x for x in self.product_list if x['sn'][0]=='F']
        else:
            search_list = self.product_list

        # 把辨識描述用word2vec轉成300為的詞向量
        key_vec = np.zeros(300)
        for d in keyword:
            # 所有詞向量加總
            key_vec += self.cloth_vec[d]

        # 把產品詞向量存成矩陣
        pro_vec = np.empty((len(search_list),300))
        for i, n in enumerate(search_list):
            str_vec = n['vector']
            pro_vec[i] = np.fromstring(str_vec, sep=' ')
        
        # 找出夾角最小（最大cos）商品的索引
        dot  = np.dot(key_vec, pro_vec.T)
        norm = np.linalg.norm(key_vec) * np.linalg.norm(pro_vec, axis=1)
        cos  = dot / norm
        idx  = np.argmax(cos)
        sco  = np.max(cos)

        return (search_list[idx],
                search_list[idx]['sn'], 
                search_list[idx]['name'], 
                search_list[idx]['image_url'],
                round(sco, 3))
        
    # 輸入商品節點回傳同款商品
    def same_search(self, product):
        same_relate = list(neo.RelationshipMatch(self.graph, 
                                                 nodes=[product], 
                                                 r_type='SAME'))
        same_node = same_relate[0].end_node
        if same_node != []:
            return (same_node,
                    same_node['sn'],
                    same_node['name'],
                    same_node['image_url'])
        else:
            return '無同款'   

    def fit_search(self, product, top=1):
        fit_relate = neo.RelationshipMatch(self.graph, nodes=[product], r_type='FIT')
        fit_rank = list(fit_relate.order_by('_.score DESC'))
        fit_score = fit_rank[top-1]['score']
        fit_node = fit_rank[top-1].end_node
        return (fit_node, 
                fit_score,
                fit_node['sn'],
                fit_node['name'],
                fit_node['image_url'])


### 範例

In [64]:
# 建立物件
sever_link = 'bolt://52.91.118.4:33083'
pws = 'sevenths-harpoon-jails'
word2vec_path = 'cloth_word_vec.csv'

recommdation = Neo4jRecomBot('bolt://52.91.118.4:33083', 'sevenths-harpoon-jails', word2vec_path)

recommdation.graph

Graph('bolt://neo4j@52.91.118.4:33083')

### 模糊搜尋

In [65]:
# 隨機生成特徵描述

keyword = []
word_list = list(recommdation.cloth_vec)
word_num = len(cloth_vec)

num = 3
ridx = np.random.randint(0, word_num, num)


for i in ridx:
    word = word_list[i]
    keyword.append(word)

keyword

['馬褲', '睡衣', '長袖']

In [66]:
gender = 'woman'

main_recom, main_sn, main_name, main_url, match_score = recommdation.fussy_search(keyword, gender=gender)

main_name, match_score

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.


('Soft RichV領針織長上衣', 0.076)

### 同款搜尋

In [ ]:
recommdation.same_search(main_recom)

(Node('Product', description='窄管褲剪裁的輕便褲。採用外觀乾淨的設計，外出穿也方便。使用觸感柔軟舒適的平織素材，讓穿著感更舒適。腰部和腳踝有高密度羅紋，更合身。米白色', image_url='https://im.uniqlo.com/images/tw/gu/pc/goods/325118/item/30_325118.jpg', name='休閒輕便褲', sn='FL13', vector='0.09307863935828209-0.0455837871413677919.5698366761207583.093698922544718-0.7791512816911563-0.23970863455906510.11401115567423403-4.12436121713835756.2972416223492471.5279800430871546-0.26323886972386390.007235851720906794-1.1236204034648836-0.324919049628078942.01073746895417571.3811230235733092-0.6537337615154684-0.90783396502956751.31158485542982820.7908976235194132-0.67894496207009070.0093757624272257094.735925829038024-1.62569858774077150.33998557110317050.5427217296091840.7329323776066303-1.1407209541357588-0.49879218230489640.020300468197092414-1.45949497818946841.21533414657460531.55052169092232360.8288054049480706-1.47226393106393522.2134980529081076-0.11595311807468534-5.660951932892203-3.73177644051611421.0469575031602290.17307529901154340.107715423218905930.8

In [ ]:
same_recom, same_sn, same_name, same_url = recommdation.fussy_search(keyword, gender='woman')

same_name

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.


'Soft RichV領針織長上衣'

### 穿搭推薦

In [51]:
# 一階穿搭
fit_recom_first, fit_score_f, fit_sn_f, fit_name_f, fit_url_f = recommdation.fit_search(main_recom, top=1)

print('一階穿搭:', fit_name_f, '分數:', fit_score_f)

一階穿搭: 寬版高領針織長上衣 分數: 0.817


In [52]:
# 二階穿搭
fit_recom_second, fit_score_s, fit_sn_s, fit_name_s, fit_url_s = recommdation.fit_search(fit_recom_first, top=2)

print('二階穿搭:', fit_name_s, '分數:', fit_score_s)

二階穿搭: 夾層寬版大衣 分數: 0.801
